In [1]:
import instaloader
import pandas as pd
import nltk
import pytesseract
import malaya

c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\malaya\tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\malaya\tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
L = instaloader.Instaloader()

USERNAME = "faizgram"
SESSION_FILE = "session-faizgram"

try:
    L.load_session_from_file(username=USERNAME, filename=SESSION_FILE)
except FileNotFoundError as e:
    print("Session file not found. Please log in manually using Instaloader CLI.")
    exit()
except instaloader.ConnectionException as e:
   print(f"Connection exception {e}")
   exit()

Loaded session from session-faizgram.


In [3]:
def by_hashtag(hashtag, limit, profile_name=None, location=None):
    post_count = 0
    hashtag_posts = pd.DataFrame(columns=["Thumb", "Caption", "Comment"])
    thumbs = []
    captions = []
    comments = []
    dates = []

    posts = L.get_hashtag_posts(hashtag)

    for post in posts:
        if post.caption:
            thumbs.append(post.url)
            captions.append(post.caption)
            comments.append(post.comments)
            dates.append(post.date)
            # print(f"Extracted caption: {post.caption}")
        
        post_count += 1
        if post_count >= limit:
            break

    hashtag_posts["Thumb"] = thumbs
    hashtag_posts["Caption"] = captions
    hashtag_posts["Date"] = dates
    hashtag_posts["Comment"] = comments

    return hashtag_posts

In [4]:
posts = by_hashtag("unikl", 5, None, None)

In [5]:
posts

,Thumb,Caption,Comment,Date
0,https://instagram.fkul4-4.fna.fbcdn.net/v/t51....,CONVO DAY IPG MALAYSIA 24!\nKONVO DAH NAK DEK...,0,2024-12-10 16:28:40
1,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,CONVO DAY IPG MALAYSIA 24!\nKONVO DAH NAK DEK...,0,2024-12-10 16:22:35
2,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,Train SCS Class 92 🚝\n\nGambar sekitar Majlis ...,0,2024-12-10 15:16:46
3,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,❤️❤️RUN❤️❤️\nSouthern Bundle Collection\n\nNB ...,0,2024-12-10 13:39:19
4,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nA...,0,2024-12-10 12:35:53


### Preprocessing

In [6]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from transformers import pipeline

from PIL import Image
import requests
from io import BytesIO

# Tesseract path (Windows 10/11)
tesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Unix/Linux

# MacOS

pytesseract.pytesseract.tesseract_cmd = tesseract_path

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
img_texts = []
for img_url in posts["Thumb"]:
    response = requests.get(img_url)
    img_text = pytesseract.image_to_string(Image.open(BytesIO(response.content)))
    print(f"Extracted text: {img_text}")
    img_texts.append(img_text)

posts["Thumb Text"] = img_texts

Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: 


In [8]:
posts

,Thumb,Caption,Comment,Date,Thumb Text
0,https://instagram.fkul4-4.fna.fbcdn.net/v/t51....,CONVO DAY IPG MALAYSIA 24!\nKONVO DAH NAK DEK...,0,2024-12-10 16:28:40,
1,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,CONVO DAY IPG MALAYSIA 24!\nKONVO DAH NAK DEK...,0,2024-12-10 16:22:35,
2,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,Train SCS Class 92 🚝\n\nGambar sekitar Majlis ...,0,2024-12-10 15:16:46,
3,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,❤️❤️RUN❤️❤️\nSouthern Bundle Collection\n\nNB ...,0,2024-12-10 13:39:19,
4,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nA...,0,2024-12-10 12:35:53,


In [20]:
def preprocess_text(text):
    # this uses NLTK
    # tokens = word_tokenize(text.lower())

    # this uses malaya toolkit
    tokenizer = malaya.tokenizer.Tokenizer()    
    tokens = tokenizer.tokenize(text, lowercase=True)

    # remove alphanumeric word(s)
    filtered_tokens = [word for word in tokens if word.isalnum()]
    
    return ' '.join(filtered_tokens)

In [22]:
texts = []
for text in posts["Caption"]:
   texts.append(preprocess_text(text))

texts

['convo day ipg malaysia 24 konvo dah nak dekat dah kenangan bersama sahabat wajib ada untuk kali terakhirr jom tempah photographer convobyzown promotion harga sekarang officials photographer lihat hasil shoot konvo di gambar anda akan diberikan melalui telegram tau',
 'convo day ipg malaysia 24 konvo dah nak dekat dah kenangan bersama sahabat wajib ada untuk kali terakhirr jom tempah photographer convobyzown promotion harga sekarang officials photographer lihat hasil shoot konvo di gambar anda akan diberikan melalui telegram tau',
 'train scs class 92 gambar sekitar majlis penyerahan handback ceremony tren scs class 92 projek maintenance repair overhaul mro dan perjanjian kolaborasi collaboration agreement di antara m rail technics railtec dengan universiti kuala lumpur unikl oleh yab dato seri ahmad zahid bin hamidi timbalan perdana menteri malaysia',
 'run southern bundle collection nb 411 all black so ringan size 9uk kondisi rm 128 free pos semenanjung',
 'court southern bundle col

### Sentiment Analysis

In [ ]:
# load model
sent_model = malaya.sentiment.multinomial()

sentiments = []
sentiment_probas = []
for text in texts:
    sentiments.append(sent_model.predict(text))
    sentiment_probas = (sent_model.predict_proba(text))

posts["Sentiment"] = sentiments
posts["Probability"] = sentiment_probas

multinomial.pkl:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohdf\.cache\huggingface\hub\models--huseinzol05--v47-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tfidf.pkl:   0%|          | 0.00/2.72M [00:00<?, ?B/s]

bpe.model:   0%|          | 0.00/875k [00:00<?, ?B/s]

c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ComplementNB from version 0.22.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.1 when using version 1.6

ModuleNotFoundError: youtokentome not installed. Please install it by `pip install youtokentome` and try again.

### Emotion Analysis

In [30]:
model = malaya.emotion.multinomial()

for text in texts:
    model.predict(text)

multinomial.pkl:   0%|          | 0.00/6.24M [00:00<?, ?B/s]

c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohdf\.cache\huggingface\hub\models--huseinzol05--v34-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tfidf.pkl:   0%|          | 0.00/2.72M [00:00<?, ?B/s]

bpe.model:   0%|          | 0.00/881k [00:00<?, ?B/s]

c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ComplementNB from version 0.22.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mohdf\anaconda\envs\work\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.1 when using version 1.6

ModuleNotFoundError: youtokentome not installed. Please install it by `pip install youtokentome` and try again.